Import libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime

Set variables

In [5]:
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'

Define the log function

In [3]:
def log_progress(message):
    now = datetime.now().strftime('%Y-%h-%d-%H:%M:%S')
    with open("./code_log.txt","a") as f:
        f.write(now + ' : ' + message + '\n')

Define the extract function
The function receives the url and the columns of the table and returns a dataframe

In [72]:
def extract(url, table_attribs):
    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col) != 0:
            data_dict = {"Rank" : col[0].text[0],
                         "Bank Name" : col[1].find_all('a')[1].text,
                         "Market Cap" : col[2].text.strip('\n')}
            print(data_dict)
            #print(col[1].find_all('a')[1].text)

In [73]:
extract(url, ['Rank','Bank Name','Market Cap'])

{'Rank': '1', 'Bank Name': 'JPMorgan Chase', 'Market Cap': '432.92'}
{'Rank': '2', 'Bank Name': 'Bank of America', 'Market Cap': '231.52'}
{'Rank': '3', 'Bank Name': 'Industrial and Commercial Bank of China', 'Market Cap': '194.56'}
{'Rank': '4', 'Bank Name': 'Agricultural Bank of China', 'Market Cap': '160.68'}
{'Rank': '5', 'Bank Name': 'HDFC Bank', 'Market Cap': '157.91'}
{'Rank': '6', 'Bank Name': 'Wells Fargo', 'Market Cap': '155.87'}
{'Rank': '7', 'Bank Name': 'HSBC Holdings PLC', 'Market Cap': '148.90'}
{'Rank': '8', 'Bank Name': 'Morgan Stanley', 'Market Cap': '140.83'}
{'Rank': '9', 'Bank Name': 'China Construction Bank', 'Market Cap': '139.82'}
{'Rank': '1', 'Bank Name': 'Bank of China', 'Market Cap': '136.81'}
